In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

TO DO:
Test oversampling
Test undersampling
SVMs w optimizations
Feature select the coeffs

Loading the data in

In [29]:
# This is basically the numerical representation of the audio file
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data,2,0)
#Read the df
df = pd.read_csv('primary_dataframe.csv')
y = df.diagnosis_Healthy
del df # Get this out of memory

In [30]:
# Load the data
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data, 2, 0)

# Create a list to store the data
data_list = []

# Loop over each audio file
for i in range(len(data)):
    # Loop over each time step
    time_steps = []
    for j in range(1, len(data[i])):
        time_steps.append(data[i][j])
    
    # Append the data for this audio file to the list
    data_list.append(time_steps)

data_list is a list of 920 arrays each containing one value for each index.
so data_list[record][coeff][timestep] is basically the access key

I want to format the df where its 862*39 columns each column should represent a timepoint and its coeff.
For example coeff1_1, coeff1_2 etc

In [31]:
#Expanded data list
expanded_list = []
pt = []
for data_pt in data_list:
    expanded_list.append(pt)
    pt = []
    for coeff in data_pt:
        for time_point in coeff:
            pt.append(time_point)

#Take the first element out of expanded_list
expanded_list = expanded_list[1:]

In [32]:
df = pd.DataFrame(expanded_list)
del expanded_list # Get this out of memory
#Add the label
df['label'] = y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,33609,33610,33611,33612,33613,33614,33615,33616,33617,label
0,0.0,0.0,0.0,121.746948,136.982208,130.102921,124.637711,126.339783,121.489182,114.666153,...,1.049468,3.614823,0.639579,-1.597457,1.816956,2.416618,-0.118036,3.692537,5.124985,0.0
1,0.0,0.0,0.0,115.861801,118.390411,118.370117,109.110718,109.259323,111.860153,113.856010,...,-0.227390,3.137236,1.418419,3.772516,7.746039,3.535635,-1.156423,0.290763,0.832430,0.0
2,0.0,0.0,0.0,103.759872,116.726585,131.779861,133.090576,128.802948,128.088867,131.850586,...,1.426593,1.742175,-0.133874,-0.698511,2.769688,6.254745,2.060101,2.717711,7.071089,0.0
3,0.0,0.0,0.0,84.588654,116.151443,118.165756,127.619461,130.909302,124.239517,117.667412,...,-1.887370,0.797615,2.225058,6.033323,9.317051,5.597347,6.741650,10.329792,5.493144,0.0
4,0.0,0.0,0.0,112.923431,114.193184,113.110992,97.997498,98.535042,96.229752,98.645630,...,10.215909,1.997435,-2.325577,3.389224,7.490587,5.754083,3.451414,2.424506,1.807187,0.0


Under Sampling:

In [50]:
# Count the number of rows with label 0 and 1
num_0 = df['label'].value_counts()[0]
num_1 = df['label'].value_counts()[1]


# Calculate the number of rows to remove
num_to_remove = abs(num_0 - num_1)-100

# Remove rows with the minimum label until the number of rows for each label is equal
df_removed = df.drop(df[df['label'] == 0].sample(num_to_remove).index)

In [51]:
from sklearn.model_selection import train_test_split

# Extract the data and labels from the dataframe
X = df_removed.loc[:, ~df_removed.columns.isin(['label'])]
y = df_removed['label'].values

# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31, stratify=y)

In [53]:
# PCA to reduce the dimensionality of the data
from sklearn.decomposition import PCA

#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a PCA object to transform the data to an adequate amount judged by elbows
pca = PCA(n_components=0.95, random_state=31)
# Do the PCA
pca.fit(X_train)
# Transform the data
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [55]:
from sklearn.svm import SVC

# Create an SVM model with a linear kernel
model = SVC(kernel='linear')

# Train the model on the training data
model.fit(X_train, y_train)

# Evaluate the model on the test data
acc = model.score(X_test, y_test)
print(f"Accuracy: {acc}")

Accuracy: 0.8529411764705882


In [56]:
from sklearn.metrics import classification_report

# Evaluate the model on the test data and print precision, recall, and F1-score
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Create a labelled confusion matrix
cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
print(cm)

              precision    recall  f1-score   support

         0.0       0.87      0.96      0.91        27
         1.0       0.75      0.43      0.55         7

    accuracy                           0.85        34
   macro avg       0.81      0.70      0.73        34
weighted avg       0.84      0.85      0.84        34

Predicted  0.0  1.0
Actual             
0.0         26    1
1.0          4    3


Normal Sampling:

In [57]:
from sklearn.model_selection import train_test_split

# Extract the data and labels from the dataframe
X = df.loc[:, ~df.columns.isin(['label'])]
y = df['label'].values

# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31, stratify=y)

In [58]:
# PCA to reduce the dimensionality of the data
from sklearn.decomposition import PCA

#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a PCA object to transform the data to an adequate amount judged by elbows
pca = PCA(n_components=0.95, random_state=31)
# Do the PCA
pca.fit(X_train)
# Transform the data
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [59]:
from sklearn.svm import SVC

# Create an SVM model with a linear kernel
model = SVC(kernel='rbf')

# Train the model on the training data
model.fit(X_train, y_train)

# Evaluate the model on the test data
acc = model.score(X_test, y_test)
print(f"Accuracy: {acc}")

Accuracy: 0.9619565217391305


In [60]:
from sklearn.metrics import classification_report

# Evaluate the model on the test data and print precision, recall, and F1-score
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Create a labelled confusion matrix
cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
print(cm)

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       177
         1.0       0.00      0.00      0.00         7

    accuracy                           0.96       184
   macro avg       0.48      0.50      0.49       184
weighted avg       0.93      0.96      0.94       184

Predicted  0.0
Actual        
0.0        177
1.0          7


/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf